<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

Check SQL Version

In [2]:
import sqlite3
ver = sqlite3.sqlite_version_info
print("SQLite Version {}.{}.{}".format(ver[0], ver[1], ver[2]))

SQLite Version 3.40.1


And then we import our movie database

In [3]:
%sql sqlite:///theaters.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [4]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///theaters.sqlite
Done.


[]

## SQL statements to review at the lab session

Write SQL code for the following tasks:

+ Show the names of all movies.

In [5]:
%%sql
SELECT MovieTitle
FROM   movie

 * sqlite:///theaters.sqlite
Done.


MovieTitle
Avatar: The Way of Water
Forest Gump
Silence of the Lambs
The Dark Night
The Breakfast Club
The Godfather
Top Gun
Mission Impossible - Fallout
Ponyo
Blade Runner 2049


+ Show the performance dates for one of the movies.

In [6]:
%%sql
SELECT    MovieTitle, PerformanceId, StartTime, PerformanceDate, TheaterName
FROM      movie
LEFT JOIN performance
USING     (IMDBKey)
WHERE     MovieTitle="The Terminator"

 * sqlite:///theaters.sqlite
Done.


MovieTitle,PerformanceId,StartTime,PerformanceDate,TheaterName
The Terminator,17,21:00,2022-11-06,Astoria
The Terminator,18,22:20,2022-11-06,Maximteatern


+ Show all data concerning performances at a given theatere
  on a given date.

In [7]:
%%sql
SELECT      *
FROM        movie
RIGHT JOIN  performance
USING       (IMDBKey)
RIGHT JOIN  theater
USING       (TheaterName)
WHERE       PerformanceDate="2022-12-19" AND TheaterName="Paramount Theater"

 * sqlite:///theaters.sqlite
Done.


IMDBKey,MovieTitle,ProductionYear,RunningTime,PerformanceId,StartTime,PerformanceDate,TheaterName,Capacity
tt0092099,Top Gun,1986,109,10,18:15,2022-12-19,Paramount Theater,12
tt0092099,Top Gun,1986,109,11,21:20,2022-12-19,Paramount Theater,12


+ List all customers

In [8]:
%%sql
SELECT *
FROM   customer

 * sqlite:///theaters.sqlite
Done.


Username,CustomerName,UserPassword
test123,John Smith,password123
brian22,Brian Svensson,SeCURePassword4$!
alice2002,Alice Johnson,a53g4hj7ks5b6hb!#$#
jacob1576,Jacob Krucinski,moVieAccount7428@


+ List all tickets

In [9]:
%%sql
SELECT *
FROM   ticket

 * sqlite:///theaters.sqlite
Done.


TicketId,PerformanceId,Username
757be03a362d7f90e28490730e2c014c,1,test123
95ed6ec0fe370bb6bcac5456b78bc1fa,2,test123
347bba659a6744f3f25c4cf012fe548e,3,test123
b88887153900dbff9dd6f82eb75587d6,3,jacob1576
62390aff22b642c3ee04e2ee071acb6f,3,jacob1576
0cc22e9084151bd9c1452ec009d79709,4,alice2002
71624c294e4dae2afd6ed07772dad948,5,jacob1576
bf5adbd0569c1b7ae96a91c4fb7cab5c,19,brian22
1bd0c98ed6e4602fedee40d6107af85c,12,test123
c1c394d284b9835c1215f18aace932fb,13,test123


## Things to try out yourselves (not reviewed at the lab session)

During lab 3 you'll write a REST service for the database
you just created, and for the server to work properly, the
things below should work.

If you have any questions about the material below, feel
free to ask them at the QA sessions (see Moodle for how to
sign up for them).


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [10]:
%%sql
INSERT
INTO        Ticket(Username, PerformanceId)
VALUES      ("alice2002", 2)
RETURNING   TicketId;

 * sqlite:///theaters.sqlite
(sqlite3.OperationalError) cannot commit transaction - SQL statements in progress
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [11]:
def insert_ticket(username, performance_id):
    # Regular SQL failing, try using sqlite3 library instead
    db = sqlite3.connect("theaters.sqlite")
    c = db.cursor()
    c.execute("PRAGMA foreign_keys = 1")
    c.execute("""INSERT INTO Ticket(Username, PerformanceId) VALUES (?, ?) RETURNING TicketId""", 
              [username, performance_id])

    found = c.fetchone()
    if not found:
        raise ValueError
    else:
        db.commit()
        c.close() 
        return found[0]

In [12]:
def insert_theater(name, capacity):
    # Regular SQL failing, try using sqlite3 library instead
    db = sqlite3.connect("theaters.sqlite")
    c = db.cursor()
    c.execute("PRAGMA foreign_keys = 1")
    c.execute("""INSERT INTO Theater(TheaterName, Capacity) VALUES (?, ?) RETURNING TheaterName""", 
              [name, capacity])

    print("executed")
    found = c.fetchone()
    print(found)
    if not found:
        raise ValueError
    else:
        db.commit()
        c.close() 
        return found[0]

In [13]:
def insert_performance(perf_id, time, date, theater_name, imdb_key):
    # Regular SQL failing, try using sqlite3 library instead
    db = sqlite3.connect("theaters.sqlite")
    c = db.cursor()
    c.execute("""INSERT INTO Performance(PerformanceId, StartTime, PerformanceDate, TheaterName, IMDBKey) 
                 VALUES (?, ?, ?, ?, ?) RETURNING IMDBKey""", 
                 [perf_id, time, date, theater_name, imdb_key])

    print("executed")
    found = c.fetchone()
    if not found:
        raise ValueError
    else:
        db.commit()
        c.close() 
        return found[0]

In [14]:
insert_id = insert_ticket("alice2002", 2)
print(insert_id)

819df13732b20261e1b1369f07e423cc


In [15]:
%%sql
SELECT *
FROM   ticket

 * sqlite:///theaters.sqlite
Done.


TicketId,PerformanceId,Username
757be03a362d7f90e28490730e2c014c,1,test123
95ed6ec0fe370bb6bcac5456b78bc1fa,2,test123
347bba659a6744f3f25c4cf012fe548e,3,test123
b88887153900dbff9dd6f82eb75587d6,3,jacob1576
62390aff22b642c3ee04e2ee071acb6f,3,jacob1576
0cc22e9084151bd9c1452ec009d79709,4,alice2002
71624c294e4dae2afd6ed07772dad948,5,jacob1576
bf5adbd0569c1b7ae96a91c4fb7cab5c,19,brian22
1bd0c98ed6e4602fedee40d6107af85c,12,test123
c1c394d284b9835c1215f18aace932fb,13,test123


In [16]:
%%sql
-- Delete the entry from above
DELETE
FROM   Ticket
WHERE  TicketId="085c057c255978d73779c347951ed073"

 * sqlite:///theaters.sqlite
0 rows affected.


[]

In [17]:
%%sql
SELECT *
FROM   ticket

 * sqlite:///theaters.sqlite
Done.


TicketId,PerformanceId,Username
757be03a362d7f90e28490730e2c014c,1,test123
95ed6ec0fe370bb6bcac5456b78bc1fa,2,test123
347bba659a6744f3f25c4cf012fe548e,3,test123
b88887153900dbff9dd6f82eb75587d6,3,jacob1576
62390aff22b642c3ee04e2ee071acb6f,3,jacob1576
0cc22e9084151bd9c1452ec009d79709,4,alice2002
71624c294e4dae2afd6ed07772dad948,5,jacob1576
bf5adbd0569c1b7ae96a91c4fb7cab5c,19,brian22
1bd0c98ed6e4602fedee40d6107af85c,12,test123
c1c394d284b9835c1215f18aace932fb,13,test123


In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [18]:
%%sql
SELECT TicketId
FROM   Ticket
WHERE  rowid = last_insert_rowid();

 * sqlite:///theaters.sqlite
Done.


TicketId
819df13732b20261e1b1369f07e423cc


+ Try to insert two movie theaters with the same name (this
  should fail).

In [19]:
%%sql
--Reset if necessary
DELETE
FROM   Theater
WHERE  TheaterName="Lund Theater"

 * sqlite:///theaters.sqlite
0 rows affected.


[]

In [20]:
# Uncomment the 2 lines below to test
#insert_theater("Lund Theater", 200)
#insert_theater("Lund Theater", 100)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [22]:
%%sql
--Reset if necessary
DELETE
FROM   Performance
WHERE  PerformanceId=20

 * sqlite:///theaters.sqlite
(sqlite3.OperationalError) database is locked
[SQL: --Reset if necessary
DELETE
FROM   Performance
WHERE  PerformanceId=20]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [21]:
# Why didn't this fail??
insert_performance(20,"15:00","2023-02-05","Nonexistant Theater","tt1630029")

IntegrityError: UNIQUE constraint failed: Performance.PerformanceId

+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [ ]:
%%sql
